## Import libraries


In [1]:
import numpy as np
import pandas as pd
from random import choice, uniform, random, randint
from operator import itemgetter, attrgetter
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Activation,Flatten,Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras import backend
from tensorflow.keras.applications import VGG16, MobileNet
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display
import random
import math
import tensorflow as tf
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend, models
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import VGG16, MobileNet
from keras.applications.vgg16 import preprocess_input

In [2]:
population = []
noPop = 6
fmap = [16,32,64,128,256]
kernel = [2,3,5,7,9]
activation = ['relu','sigmoid','tanh']
dropout = [0.0,0.1,0.2,0.3,0.4,0.5]
learningRate = [0.0001,0.0005, 0.001, 0.005, 0.01] 
mutationRate = 0.3 
noGen = 5 

## Preparation and analysis of dataset

In [3]:
np.random.seed(12)
train = ImageDataGenerator(rescale=1./255).flow_from_directory('dataset/train',target_size=(224,224))

Found 35215 images belonging to 250 classes.


## Genetic Algorithm

In [4]:
def initializingPopulation(population,noPop):
    for i in range (0,noPop):
        loss = 0.0
        accuracy = 0.0
        inPop = [choice(fmap),choice(kernel),choice(activation),choice(dropout),choice(learningRate),loss,accuracy]
        population.append (inPop)

def rankSelection (population,noPop): 
    population = sorted(population,key=itemgetter(5))
    population = sorted(population,key=itemgetter(6),reverse=True)
    noHalfPopulation = noPop//2 
    halfPopulation = population[:noHalfPopulation]

    return halfPopulation

def tournamentSelection(selectedRank,tSize):
    best = None 
    for i in range(0,tSize):
        ind = choice(selectedRank)
        if best == None:
            best = ind
        elif best == ind:
            ind = choice(selectedRank)

        if best[5]>ind[5] and best[6]<ind[6]:
            best = ind

    return best

def crossover(p1,p2): 
    crossoverPoint = len(p1)//2 
    c1 = p1[:crossoverPoint] + p2[crossoverPoint:]
    c2 = p2[:crossoverPoint] + p1[crossoverPoint:]
    return c1, c2

def mutation (c1,c2):
    rate = random.random()
    print(c1)
    print(c2)
    
    print ('rate: ' +str(rate))
    if rate<=mutationRate:
        length = len(c1)-2
        print ('length: '+str(length))
        index = randint(0,length)
        print('index: ' +str(index))

        if index == 0:
            c1[index] = choice(fmap)
            c2[index] = choice(fmap)
        elif index == 1:
            c1[index] = choice(kernel)
            c2[index] = choice(kernel)
        elif index == 2:
            c1[index] = choice(activation)
            c2[index] = choice(activation)
        elif index == 3:
            c1[index] = choice(dropout)
            c2[index] = choice(dropout)
        elif index == 4:
            c1[index] = choice(learningRate)
            c2[index] = choice(learningRate)
    return c1,c2

In [5]:
def cnn(population):
    for p in population:
        #this is where the fitness value will evaluate
        f = p[0] # features map variable
        k = p[1] # kernel variable
        a = p[2] # activation function variable
        d = p[3] # dropout variable
        l = p[4] # learning rate variable
        print(f,k,a,d,l)
        
        backend.clear_session()

        base_mobilenet = MobileNet(weights = 'imagenet', include_top = False, 
                           input_shape = (224,224,3))
        base_mobilenet.trainable = False # Freeze the mobilenet weights.

        model = Sequential()
        model.add(base_mobilenet)
        model.add(Conv2D(f,(k,k),input_shape=(224,224,3),padding='same'))
        model.add(Activation(a))
        model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
        model.add(Flatten())
        model.add(Dense(32*32))
        model.add(Dropout(d))
        model.add(Dense(250))
        model.add(Activation('softmax'))

        model.compile(Adam(l),loss='categorical_crossentropy',metrics=['accuracy'])

        history = model.fit(train,epochs=100,steps_per_epoch=10,verbose=2)
        acc = np.mean(history.history['accuracy'])
        loss = np.mean(history.history['loss'])
        print(acc)
        print(loss)

        p[5] = loss
        p[6] = acc
        print(p[5])
        print(p[6])


initializingPopulation(population,noPop)

print ('Generation 0')
for p in population:
    print(p)

for gen in range(0,noGen):

    cnn(population)

    selectedRank = rankSelection(population,noPop)
    print('Selected Ranked Chromosomes:')
    for sr in selectedRank:
        print(sr)

    if selectedRank[-1][5]<6 and selectedRank[-1][6]>0.6:
        # this is  stopping condition for genetic algorithm.
        # you can change or delete it if you want
        # but, remember, waiting for 10 generation is time consumption
        # and also computationally overhead
        break
    else:
        child = [] 
        tSize = 2 
        numberOfMating = len(selectedRank)//2
        # this is limiter  for number of child will generate
        # let say, you have 12 total population
        # let say, you have 6 ranking selection
        # the mating process will be 3 time
        # but, since the generated child is 2
        # thus mating process would generate 6 children
        # this is because number of mating * generated Children = half new population
        # 3 mating process * 2 newly generated child = 6 population
        for i in range (0,numberOfMating):
            #this is where mating process take place
            parent1 = tournamentSelection(selectedRank,tSize)
            parent2 = tournamentSelection(selectedRank,tSize)
            while parent2 is parent1:
                parent2 = tournamentSelection(selectedRank,tSize)
                #this is to make sure the candidate does not identical
            c1,c2 = crossover(parent1,parent2)
            print('Child 1: %s child 2: %s'%(c1,c2))
            c1,c2 = mutation(c1,c2)
            child.append(c1)
            child.append(c2)

        population.clear() 
        population = selectedRank + child 

        print ('New Child' + str(gen))
        for c in child:
            print(c)

        print('New Generation ' + str(gen+1))
        for p in population:
            print(p)

Generation 0
[64, 7, 'tanh', 0.2, 0.0001, 0.0, 0.0]
[128, 2, 'tanh', 0.4, 0.01, 0.0, 0.0]
[16, 5, 'tanh', 0.0, 0.01, 0.0, 0.0]
[32, 3, 'tanh', 0.2, 0.01, 0.0, 0.0]
[128, 5, 'relu', 0.1, 0.0001, 0.0, 0.0]
[32, 2, 'sigmoid', 0.4, 0.01, 0.0, 0.0]
64 7 tanh 0.2 0.0001
Epoch 1/100
10/10 - 5s - loss: 6.1539 - accuracy: 0.0125
Epoch 2/100
10/10 - 4s - loss: 5.8619 - accuracy: 0.0063
Epoch 3/100
10/10 - 4s - loss: 5.7327 - accuracy: 0.0281
Epoch 4/100
10/10 - 4s - loss: 5.3952 - accuracy: 0.0437
Epoch 5/100
10/10 - 4s - loss: 5.4443 - accuracy: 0.0531
Epoch 6/100
10/10 - 4s - loss: 5.3996 - accuracy: 0.0531
Epoch 7/100
10/10 - 4s - loss: 5.1386 - accuracy: 0.0500
Epoch 8/100
10/10 - 4s - loss: 4.8591 - accuracy: 0.1000
Epoch 9/100
10/10 - 4s - loss: 4.7076 - accuracy: 0.1156
Epoch 10/100
10/10 - 4s - loss: 4.6850 - accuracy: 0.1031
Epoch 11/100
10/10 - 4s - loss: 4.5269 - accuracy: 0.1500
Epoch 12/100
10/10 - 4s - loss: 4.3325 - accuracy: 0.1625
Epoch 13/100
10/10 - 4s - loss: 4.0981 - accurac

Epoch 35/100
10/10 - 3s - loss: 268.6379 - accuracy: 0.0281
Epoch 36/100
10/10 - 3s - loss: 275.7770 - accuracy: 0.0156
Epoch 37/100
10/10 - 3s - loss: 272.7013 - accuracy: 0.0219
Epoch 38/100
10/10 - 3s - loss: 290.8650 - accuracy: 0.0375
Epoch 39/100
10/10 - 3s - loss: 295.4000 - accuracy: 0.0219
Epoch 40/100
10/10 - 3s - loss: 300.0400 - accuracy: 0.0375
Epoch 41/100
10/10 - 3s - loss: 299.3231 - accuracy: 0.0219
Epoch 42/100
10/10 - 3s - loss: 295.0790 - accuracy: 0.0219
Epoch 43/100
10/10 - 3s - loss: 287.8328 - accuracy: 0.0312
Epoch 44/100
10/10 - 3s - loss: 268.7394 - accuracy: 0.0312
Epoch 45/100
10/10 - 3s - loss: 289.3368 - accuracy: 0.0469
Epoch 46/100
10/10 - 3s - loss: 245.6535 - accuracy: 0.0562
Epoch 47/100
10/10 - 3s - loss: 252.9006 - accuracy: 0.0656
Epoch 48/100
10/10 - 3s - loss: 250.5663 - accuracy: 0.0156
Epoch 49/100
10/10 - 3s - loss: 271.3756 - accuracy: 0.0344
Epoch 50/100
10/10 - 3s - loss: 246.4645 - accuracy: 0.0656
Epoch 51/100
10/10 - 3s - loss: 251.5301

Epoch 69/100
10/10 - 3s - loss: 198.3459 - accuracy: 0.0031
Epoch 70/100
10/10 - 3s - loss: 226.5755 - accuracy: 0.0000e+00
Epoch 71/100
10/10 - 3s - loss: 272.2999 - accuracy: 0.0063
Epoch 72/100
10/10 - 3s - loss: 225.0465 - accuracy: 0.0063
Epoch 73/100
10/10 - 3s - loss: 237.7550 - accuracy: 0.0031
Epoch 74/100
10/10 - 3s - loss: 220.5869 - accuracy: 0.0031
Epoch 75/100
10/10 - 3s - loss: 237.9271 - accuracy: 0.0031
Epoch 76/100
10/10 - 3s - loss: 218.0093 - accuracy: 0.0063
Epoch 77/100
10/10 - 3s - loss: 235.1987 - accuracy: 0.0000e+00
Epoch 78/100
10/10 - 3s - loss: 229.9222 - accuracy: 0.0000e+00
Epoch 79/100
10/10 - 3s - loss: 274.1050 - accuracy: 0.0063
Epoch 80/100
10/10 - 3s - loss: 267.3639 - accuracy: 0.0125
Epoch 81/100
10/10 - 3s - loss: 257.7715 - accuracy: 0.0094
Epoch 82/100
10/10 - 3s - loss: 229.3489 - accuracy: 0.0063
Epoch 83/100
10/10 - 3s - loss: 250.7526 - accuracy: 0.0063
Epoch 84/100
10/10 - 3s - loss: 256.5663 - accuracy: 0.0031
Epoch 85/100
10/10 - 3s - lo

Epoch 3/100
10/10 - 4s - loss: 5.5842 - accuracy: 0.0156
Epoch 4/100
10/10 - 4s - loss: 5.4302 - accuracy: 0.0281
Epoch 5/100
10/10 - 4s - loss: 5.4146 - accuracy: 0.0437
Epoch 6/100
10/10 - 4s - loss: 5.3292 - accuracy: 0.0437
Epoch 7/100
10/10 - 4s - loss: 5.3408 - accuracy: 0.0406
Epoch 8/100
10/10 - 4s - loss: 5.1472 - accuracy: 0.0688
Epoch 9/100
10/10 - 4s - loss: 4.9842 - accuracy: 0.0938
Epoch 10/100
10/10 - 4s - loss: 4.8669 - accuracy: 0.1344
Epoch 11/100
10/10 - 4s - loss: 4.8657 - accuracy: 0.0969
Epoch 12/100
10/10 - 4s - loss: 4.4762 - accuracy: 0.1187
Epoch 13/100
10/10 - 4s - loss: 4.4336 - accuracy: 0.1750
Epoch 14/100
10/10 - 4s - loss: 4.1276 - accuracy: 0.2125
Epoch 15/100
10/10 - 4s - loss: 4.0608 - accuracy: 0.2031
Epoch 16/100
10/10 - 4s - loss: 3.9102 - accuracy: 0.2188
Epoch 17/100
10/10 - 4s - loss: 3.5546 - accuracy: 0.2875
Epoch 18/100
10/10 - 4s - loss: 3.2299 - accuracy: 0.3000
Epoch 19/100
10/10 - 4s - loss: 3.0025 - accuracy: 0.3625
Epoch 20/100
10/10 - 

Epoch 43/100
10/10 - 3s - loss: 5.5922 - accuracy: 0.0031
Epoch 44/100
10/10 - 3s - loss: 5.6264 - accuracy: 0.0000e+00
Epoch 45/100
10/10 - 3s - loss: 5.5500 - accuracy: 0.0156
Epoch 46/100
10/10 - 3s - loss: 5.5261 - accuracy: 0.0125
Epoch 47/100
10/10 - 3s - loss: 5.6021 - accuracy: 0.0031
Epoch 48/100
10/10 - 3s - loss: 5.5456 - accuracy: 0.0094
Epoch 49/100
10/10 - 3s - loss: 5.5391 - accuracy: 0.0156
Epoch 50/100
10/10 - 3s - loss: 5.6050 - accuracy: 0.0063
Epoch 51/100
10/10 - 3s - loss: 5.5335 - accuracy: 0.0031
Epoch 52/100
10/10 - 3s - loss: 5.5732 - accuracy: 0.0063
Epoch 53/100
10/10 - 3s - loss: 5.6201 - accuracy: 0.0063
Epoch 54/100
10/10 - 3s - loss: 5.5999 - accuracy: 0.0063
Epoch 55/100
10/10 - 3s - loss: 5.5804 - accuracy: 0.0000e+00
Epoch 56/100
10/10 - 3s - loss: 5.6184 - accuracy: 0.0000e+00
Epoch 57/100
10/10 - 3s - loss: 5.5959 - accuracy: 0.0031
Epoch 58/100
10/10 - 3s - loss: 5.5809 - accuracy: 0.0094
Epoch 59/100
10/10 - 3s - loss: 5.5772 - accuracy: 0.0125
Ep

...

Epoch 77/100
10/10 - 4s - loss: 5.5792 - accuracy: 0.0063
Epoch 78/100
10/10 - 4s - loss: 5.5996 - accuracy: 0.0063
Epoch 79/100
10/10 - 4s - loss: 5.8305 - accuracy: 0.0000e+00
Epoch 80/100
10/10 - 4s - loss: 5.5486 - accuracy: 0.0094
Epoch 81/100
10/10 - 4s - loss: 5.5330 - accuracy: 0.0125
Epoch 82/100
10/10 - 4s - loss: 5.5758 - accuracy: 0.0094
Epoch 83/100
10/10 - 4s - loss: 5.6173 - accuracy: 0.0000e+00
Epoch 84/100
10/10 - 4s - loss: 5.5834 - accuracy: 0.0063
Epoch 85/100
10/10 - 4s - loss: 5.5772 - accuracy: 0.0000e+00
Epoch 86/100
10/10 - 4s - loss: 5.5935 - accuracy: 0.0063
Epoch 87/100
10/10 - 4s - loss: 5.5913 - accuracy: 0.0000e+00
Epoch 88/100
10/10 - 4s - loss: 5.5834 - accuracy: 0.0063
Epoch 89/100
10/10 - 4s - loss: 5.6057 - accuracy: 0.0000e+00
Epoch 90/100
10/10 - 4s - loss: 5.7052 - accuracy: 0.0031
Epoch 91/100
10/10 - 4s - loss: 5.5639 - accuracy: 0.0094
Epoch 92/100
10/10 - 4s - loss: 5.5734 - accuracy: 0.0063
Epoch 93/100
10/10 - 4s - loss: 5.5802 - accuracy: 0

Epoch 1/100
10/10 - 5s - loss: 5.9524 - accuracy: 0.0188
Epoch 2/100
10/10 - 4s - loss: 5.8043 - accuracy: 0.0188
Epoch 3/100
10/10 - 4s - loss: 5.6129 - accuracy: 0.0469
Epoch 4/100
10/10 - 4s - loss: 5.4068 - accuracy: 0.0188
Epoch 5/100
10/10 - 4s - loss: 5.2360 - accuracy: 0.0469
Epoch 6/100
10/10 - 4s - loss: 5.0793 - accuracy: 0.0750
Epoch 7/100
10/10 - 4s - loss: 4.9469 - accuracy: 0.1000
Epoch 8/100
10/10 - 4s - loss: 4.7747 - accuracy: 0.1219
Epoch 9/100
10/10 - 4s - loss: 4.4897 - accuracy: 0.1906
Epoch 10/100
10/10 - 4s - loss: 4.3924 - accuracy: 0.1500
Epoch 11/100
10/10 - 4s - loss: 4.1035 - accuracy: 0.2156
Epoch 12/100
10/10 - 4s - loss: 4.0463 - accuracy: 0.1937
Epoch 13/100
10/10 - 4s - loss: 3.8221 - accuracy: 0.2625
Epoch 14/100
10/10 - 4s - loss: 3.5883 - accuracy: 0.2750
Epoch 15/100
10/10 - 4s - loss: 3.4714 - accuracy: 0.3187
Epoch 16/100
10/10 - 4s - loss: 3.4704 - accuracy: 0.2969
Epoch 17/100
10/10 - 4s - loss: 3.3906 - accuracy: 0.3344
Epoch 18/100
10/10 - 4s

Epoch 41/100
10/10 - 4s - loss: 1.3500 - accuracy: 0.6719
Epoch 42/100
10/10 - 4s - loss: 1.4678 - accuracy: 0.6719
Epoch 43/100
10/10 - 4s - loss: 1.2750 - accuracy: 0.6750
Epoch 44/100
10/10 - 4s - loss: 1.3757 - accuracy: 0.6719
Epoch 45/100
10/10 - 4s - loss: 1.3403 - accuracy: 0.6812
Epoch 46/100
10/10 - 4s - loss: 1.3306 - accuracy: 0.6719
Epoch 47/100
10/10 - 4s - loss: 1.1581 - accuracy: 0.6812
Epoch 48/100
10/10 - 4s - loss: 1.2011 - accuracy: 0.6906
Epoch 49/100
10/10 - 4s - loss: 1.1601 - accuracy: 0.7219
Epoch 50/100
10/10 - 4s - loss: 1.1130 - accuracy: 0.6969
Epoch 51/100
10/10 - 4s - loss: 1.2276 - accuracy: 0.6750
Epoch 52/100
10/10 - 4s - loss: 1.1444 - accuracy: 0.7125
Epoch 53/100
10/10 - 4s - loss: 1.0482 - accuracy: 0.6938
Epoch 54/100
10/10 - 4s - loss: 1.1559 - accuracy: 0.7250
Epoch 55/100
10/10 - 4s - loss: 1.1013 - accuracy: 0.7000
Epoch 56/100
10/10 - 4s - loss: 1.0730 - accuracy: 0.7063
Epoch 57/100
10/10 - 4s - loss: 0.9898 - accuracy: 0.7437
Epoch 58/100
1

Epoch 81/100
10/10 - 4s - loss: 0.7864 - accuracy: 0.8250
Epoch 82/100
10/10 - 4s - loss: 0.8056 - accuracy: 0.8188
Epoch 83/100
10/10 - 4s - loss: 0.8783 - accuracy: 0.7750
Epoch 84/100
10/10 - 4s - loss: 0.8922 - accuracy: 0.7875
Epoch 85/100
10/10 - 4s - loss: 0.8900 - accuracy: 0.8031
Epoch 86/100
10/10 - 4s - loss: 0.8008 - accuracy: 0.8156
Epoch 87/100
10/10 - 4s - loss: 0.9055 - accuracy: 0.8062
Epoch 88/100
10/10 - 4s - loss: 0.7758 - accuracy: 0.8219
Epoch 89/100
10/10 - 4s - loss: 0.7646 - accuracy: 0.8500
Epoch 90/100
10/10 - 4s - loss: 0.8627 - accuracy: 0.7781
Epoch 91/100
10/10 - 4s - loss: 0.6227 - accuracy: 0.8469
Epoch 92/100
10/10 - 4s - loss: 0.7053 - accuracy: 0.8406
Epoch 93/100
10/10 - 4s - loss: 0.6778 - accuracy: 0.8531
Epoch 94/100
10/10 - 4s - loss: 0.6126 - accuracy: 0.8469
Epoch 95/100
10/10 - 4s - loss: 0.5919 - accuracy: 0.8781
Epoch 96/100
10/10 - 4s - loss: 0.6506 - accuracy: 0.8438
Epoch 97/100
10/10 - 4s - loss: 0.6221 - accuracy: 0.8719
Epoch 98/100
1

10/10 - 4s - loss: 3.1353 - accuracy: 0.3500
Epoch 20/100
10/10 - 4s - loss: 3.0860 - accuracy: 0.3594
Epoch 21/100
10/10 - 4s - loss: 2.8236 - accuracy: 0.3844
Epoch 22/100
10/10 - 4s - loss: 2.6986 - accuracy: 0.4000
Epoch 23/100
10/10 - 4s - loss: 2.5526 - accuracy: 0.4344
Epoch 24/100
10/10 - 4s - loss: 2.5239 - accuracy: 0.4594
Epoch 25/100
10/10 - 4s - loss: 2.3403 - accuracy: 0.4938
Epoch 26/100
10/10 - 4s - loss: 2.4276 - accuracy: 0.4437
Epoch 27/100
10/10 - 4s - loss: 2.3046 - accuracy: 0.4531
Epoch 28/100
10/10 - 4s - loss: 2.0048 - accuracy: 0.5188
Epoch 29/100
10/10 - 4s - loss: 1.8452 - accuracy: 0.5875
Epoch 30/100
10/10 - 4s - loss: 1.8957 - accuracy: 0.5500
Epoch 31/100
10/10 - 4s - loss: 1.6832 - accuracy: 0.5875
Epoch 32/100
10/10 - 4s - loss: 1.9212 - accuracy: 0.5437
Epoch 33/100
10/10 - 4s - loss: 1.6662 - accuracy: 0.5969
Epoch 34/100
10/10 - 4s - loss: 1.6886 - accuracy: 0.5562
Epoch 35/100
10/10 - 4s - loss: 1.5652 - accuracy: 0.6156
Epoch 36/100
10/10 - 4s - l

In [6]:
print(population)

[[64, 7, 'tanh', 0.1, 0.0001, 1.8223397767543792, 0.6331485137715935], [128, 5, 'relu', 0.1, 0.0001, 1.9171594214439391, 0.5852053443947807], [64, 7, 'tanh', 0.2, 0.0001, 2.0005444157123566, 0.5915255780238659], [64, 7, 'tanh', 0.1, 0.0001, 1.8903586041927338, 0.6210340339806862], [128, 5, 'relu', 0.1, 0.0001, 1.8501557099819184, 0.6012701092893258]]
